In [49]:
serpapi_key = 'SERP_API_KEY'
open_api_key = 'OPEN_API_KEY'

In [50]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.agents import AgentType



In [120]:
# define the llm model

llm = OpenAI(openai_api_key=open_api_key, temperature=0.8) 

In [144]:
# Define your job-to-be-done and customer to be explored

customer_description = 'a single renter with average income that lives in an urban apartment'
job_to_be_done = 'monitoring and paying energy bills'

In [145]:
# Prompt hardest part of job to be done for customer
hardest_template = PromptTemplate(
    input_variables = ['customer_description','job_to_be_done'], 
    template='You are {customer_description} what is the hardest part about {job_to_be_done}?')

hardest_chain = LLMChain(llm=llm, prompt=hardest_template, verbose=True, output_key='hardest_part')

hardest = hardest_chain.run(customer_description=customer_description, job_to_be_done=job_to_be_done)

print(hardest)



> Entering new LLMChain chain...
Prompt after formatting:
You are a single renter with average income that lives in an urban appartment what is the hardest part about monitoring and paying energy bills?

> Finished chain.


The hardest part about monitoring and paying energy bills as a single renter with average income living in an urban apartment is budgeting. With the rising cost of living and rent, it can be difficult to set aside money to pay for energy bills on top of all other expenses. Additionally, it can be difficult to keep track of energy usage and bills as there is no one to share the responsibility or help remind you.


In [146]:
# Prompt value proposition to solve hardest part of job-to-be-done

value_proposition_template = PromptTemplate(
    input_variables = ['hardest_part'], 
    template='Create a unique value proposition for a startup that solves the problem of {hardest_part}')

value_proposition_chain = LLMChain(llm=llm, prompt=value_proposition_template, verbose=True, output_key='value_proposition')

valueproposition = value_proposition_chain.run(hardest_part=hardest)

print(valueproposition)




> Entering new LLMChain chain...
Prompt after formatting:
Create a unique value proposition for a startup that solves the problem of 

The hardest part about monitoring and paying energy bills as a single renter with average income living in an urban apartment is budgeting. With the rising cost of living and rent, it can be difficult to set aside money to pay for energy bills on top of all other expenses. Additionally, it can be difficult to keep track of energy usage and bills as there is no one to share the responsibility or help remind you.

> Finished chain.
 

Our startup provides a unique solution to this problem. Our service offers a streamlined and intuitive way to manage energy bills in one place. We provide users with a comprehensive dashboard containing information like energy usage and estimated bill amounts, so they can easily monitor their energy usage and budget accordingly. We also offer convenient payment options and automated bill pay, so users never miss a payment 

In [147]:
# Prompt description value proposition canvas

canvas_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Provide the value proposition canvas for {value_proposition}. You should describe the following elements: customer jobs, custome pains, customer gains, products and services, pain relievers, gain creators')

canvas_chain = LLMChain(llm=llm, prompt=canvas_template, verbose=True, output_key='canvas')

canvas = canvas_chain.run(value_proposition=valueproposition)

print(canvas)





> Entering new LLMChain chain...
Prompt after formatting:
Provide the value proposition canvas for  

Our startup provides a unique solution to this problem. Our service offers a streamlined and intuitive way to manage energy bills in one place. We provide users with a comprehensive dashboard containing information like energy usage and estimated bill amounts, so they can easily monitor their energy usage and budget accordingly. We also offer convenient payment options and automated bill pay, so users never miss a payment due date. Plus, our service offers personalized energy-saving tips to help users make the most of their energy budget. With our service, renters can easily monitor, manage, and pay their energy bills from one secure platform.. You should describe the following elements: customer jobs, custome pains, customer gains, products and services, pain relievers, gain creators

> Finished chain.
. 

Customer Jobs:
- To easily manage energy bills in one place. 
- To monitor en

In [148]:
#Prompt identification of core competitors for this value proposition

competitors_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Find existing startup companies that have a description that is very similar to the following description: {value_proposition}. Identify at least 3 existing startup companies and provide company name and location.')

competitors_chain = LLMChain(llm=llm, prompt=competitors_template, verbose=True, output_key='competitors')

competitors = competitors_chain.run(value_proposition=valueproposition)

print(competitors)



> Entering new LLMChain chain...
Prompt after formatting:
Find existing startup companies that have a description that is very similar to the following description:  

Our startup provides a unique solution to this problem. Our service offers a streamlined and intuitive way to manage energy bills in one place. We provide users with a comprehensive dashboard containing information like energy usage and estimated bill amounts, so they can easily monitor their energy usage and budget accordingly. We also offer convenient payment options and automated bill pay, so users never miss a payment due date. Plus, our service offers personalized energy-saving tips to help users make the most of their energy budget. With our service, renters can easily monitor, manage, and pay their energy bills from one secure platform.. Identify at least 3 existing startup companies and provide company name and location.

> Finished chain.


1. MyEnergy (Boston, MA)
2. OhmConnect (San Francisco, CA)
3. Enerlyti

In [150]:
# Collect additional information on competitors via SerpAPI

competitorsselect = competitors[5:]

# split string based on .

competitors_list = competitorsselect.split('\n')

competitors_list

# get informatin for competitors

search = SerpAPIWrapper(serpapi_api_key = serpapi_key)
search_tool = [Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search")]

self_ask_with_search = initialize_agent(search_tool, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

descriptionlist = []

for i in range(len(competitors_list)):
    company = competitors_list[i]
    searchstring = "Provide company information for " + company + ". Include company description, founders, funding rounds, and website."
    result = self_ask_with_search.run(searchstring)
    descriptionlist.append(result)

print(descriptionlist)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the company description of MyEnergy?
Intermediate answer: The energy sector is a category of companies that play a role in extracting, refining, or supplying consumable fuels, such as coal, oil, and gas.
MyEnergy is a full-service energy management platform dedicated to helping residential, commercial, and industrial customers reduce their energy consumption, save money, and lower their environmental impact.
Follow up: Who are the founders of MyEnergy?
Intermediate answer: Thomas Young first introduced the word “energy” to the field of physics in 1800, but the word did not gain popularity. Young later established the wave nature of light through interference experiments.

MyEnergy was founded in 2009 by Jim Marzano, former Chief Technology Officer at A123 Systems, and Maximilian Ehrlich, former Senior Engineer at the MIT Energy Initiative. 
Follow up: How many funding rounds has MyEnergy gone through?
Intermediate answer:

In [152]:
# Promp hypotheses generation for testing value proposition

hypotheses_template = PromptTemplate(
    input_variables = ['value_proposition', 'canvas'], 
    template='Generate a list of hypotheses for testing the following value proposition: {value_proposition}. You should rely on information from the value proposition canvas: {canvas}. Include at least 3 hypotheses. Each hypothesis should focus on the feasibility or viability of the business model. Here are some potential topics for hypotheses testing: willingness to pay, preferred distribution model, ultimate customer segmet, etc.')

hypotheses_chain = LLMChain(llm=llm, prompt=hypotheses_template, verbose=True, output_key='hypotheses')

hypotheses = hypotheses_chain.run(value_proposition=valueproposition, canvas=canvas)

print(hypotheses)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a list of hypotheses for testing the following value proposition:  

Our startup provides a unique solution to this problem. Our service offers a streamlined and intuitive way to manage energy bills in one place. We provide users with a comprehensive dashboard containing information like energy usage and estimated bill amounts, so they can easily monitor their energy usage and budget accordingly. We also offer convenient payment options and automated bill pay, so users never miss a payment due date. Plus, our service offers personalized energy-saving tips to help users make the most of their energy budget. With our service, renters can easily monitor, manage, and pay their energy bills from one secure platform.. You should rely on information from the value proposition canvas: . 

Customer Jobs:
- To easily manage energy bills in one place. 
- To monitor energy usage and budget accordingly. 
- To never miss a payment 

In [159]:
# Write input and output to text file

with open('customer_description.txt', 'w') as f:
    f.write('Input job to be done = ' + job_to_be_done + '\n')
    f.write('Input customer description = ' + customer_description + '\n')
    f.write('\n' + 'GPT output hardest part of job to be done = ' + hardest +  '\n')
    f.write('\n' + 'GPT output value proposition = ' + valueproposition +  '\n')
    f.write('\n' + 'GPT output value proposition canvas = ' + canvas +  '\n')
    f.write('\n' + 'GPT output most similar competitors = ' + competitors +  '\n')
    f.write('\n' + 'GPT output company description competitor 1 = ' + str(descriptionlist[0]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 2 = ' + str(descriptionlist[1]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 3 = ' + str(descriptionlist[2]) +  '\n')
    f.write('\n' + 'GPT output hypotheses = ' + hypotheses +  '\n')